In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy.ma as ma
import numpy
import datetime as dt
from datetime import datetime, timedelta
import matplotlib.cm as cm
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from palettable import colorbrewer
import numpy as np


def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError( "smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")


    s=numpy.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=numpy.ones(window_len,'d')
    else:
        w=eval('numpy.'+window+'(window_len)')

    y=numpy.convolve(w/w.sum(),s,mode='valid')
    return y


In [ ]:
adir_adcp = 'F:/data/cruise_data/saildrone/baja-2018/adcp_data/'
filename_adcp = adir_adcp + 'saildrone-gen_4-baja_2018-sd1002-adcp-20180411T175730-20180611T235730_300s-v1.nc'
ds_adcp=xr.open_dataset(filename_adcp)
ds_adcp.close()
ds_adcp = ds_adcp.isel(trajectory=0)
ds_adcp=ds_adcp.swap_dims({'obs':'time'})
print(ds_adcp)
time_offset_to_lmt=(ds_adcp.longitude/360.)*24.*60
tlmt= ds_adcp.time.data+time_offset_to_lmt*np.timedelta64(1,'m')# dt.timedelta(seconds=1)
ds_adcp['tlmt']=tlmt

In [ ]:
adir='F:/data/cruise_data/saildrone/baja-2018/'
#adir='C:/Users/gentemann/Desktop/data/'
fname=adir+'saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-10_minutes-v1_withloggers_matchedadcp_v3.nc'
#fname=adir+'saildrone-gen_4-baja_2018-EP-sd1002-ALL-1_min-v1_with_loggers.nc'
ds=xr.open_dataset(fname)
ds.close()
ds = ds.isel(obs=slice(0,8674))
ds=ds.swap_dims({'obs':'time'})
lons_usv=ds.longitude
lats_usv=ds.latitude
time_usv=ds.time
xlon = ds.longitude
ds.sea_water_temperature_logger1_mean


In [ ]:
#calculate local time
time_offset_to_lmt=(xlon/360.)*24.*60
tlmt= ds.time.data+time_offset_to_lmt*np.timedelta64(1,'m')# dt.timedelta(seconds=1)
ds['tlmt']=tlmt

In [ ]:
z=[-0.295,-0.6,-0.985,-1.42,-1.785]
i1save=[12800,35800,46000,61400,62080]
i2save=[13300,36300,46700,62080,63400]
file_str=['p1','p2','p3','p4','p5','p6','p7']
ilen=len(i1save)
ilen_ave=ds.sea_water_temperature_logger1_mean.size
tdata=np.ones([ilen_ave,5])
for i in range(0,ilen_ave):
    tdata[i,0]=ds.sea_water_temperature_logger1_mean[i]-ds.sea_water_temperature_logger4_mean[i]
    tdata[i,1]=ds.TEMP_CTD_MEAN[i]-(ds.sea_water_temperature_logger4_mean[i]) #-273.15)
    tdata[i,2]=ds.sea_water_temperature_logger2_mean[i]-ds.sea_water_temperature_logger4_mean[i]
    tdata[i,3]=ds.sea_water_temperature_logger3_mean[i]-ds.sea_water_temperature_logger4_mean[i]
    tdata[i,4]=ds.sea_water_temperature_logger4_mean[i]-ds.sea_water_temperature_logger4_mean[i]
#create xarray structure
profiles = xr.DataArray(tdata, coords={'time': ds.time,'depth':z}, dims=('time','depth'))
profiles_lmt = xr.DataArray(ds.tlmt, name='tlmt',coords={'time': ds.time}, dims=('time'))

In [ ]:
#print(profiles_lmt)
print(profiles_lmt[0].data,profiles.time[0].data)

In [ ]:
import matplotlib as mpl
from palettable import colorbrewer
import matplotlib.cm as cm
#make plots of times with diurnal warming
#add wind speed to figure
#i1save=[12800,35800,46000,61400,62000]
#i2save=[13300,36300,46700,62080,63400]
#i1save=[12800,35800,46000,61600,63080]
#i2save=[13300,36300,46700,63080,63800]
i1save=[np.datetime64('2018-04-20T06:00'),np.datetime64('2018-05-06T06:00'),np.datetime64('2018-05-13T06:00'),np.datetime64('2018-05-24T06:00'),np.datetime64('2018-05-25T06:00')]
i2save=[np.datetime64('2018-04-21T06:00'),np.datetime64('2018-05-07T06:00'),np.datetime64('2018-05-14T06:00'),np.datetime64('2018-05-25T06:00'),np.datetime64('2018-05-26T06:00')]
#i2save=[np.datetime64('2018-04-21'),np.datetime64('2018-05-07')]
                                                 

file_str=['p1','p2','p3','p4','p5','p6','p7']
ilen=len(i1save)
print(ilen)
for ii in range(0,ilen):
    i1=i1save[ii]
    i2=i2save[ii]

  #  t=tlmt.sel(time=slice(i1,i2)) #local time
  #  tem2=(t[0].values-np.datetime64('1970-01-01'))/np.timedelta64(1,'s')
  #  tem3=datetime.utcfromtimestamp(tem2)
  #  day_of_year = tem3.timetuple().tm_yday

    subset_adcp = ds_adcp.sel(time=slice(i1,i2))
    subset_adcp['vel_horiz']=np.sqrt(subset_adcp.vel_east**2+subset_adcp.vel_north**2)
    subset_adcp_masked = subset_adcp.where(subset_adcp.percent_good>10)      
    subset_usv = ds.sel(time=slice(i1,i2))
    
#    t=subset_usv.time
    t=subset_usv.tlmt
    s0=subset_usv.TEMP_CTD_MEAN.sel(time=slice(i1,i2))
    s1=subset_usv.sea_water_temperature_logger1_mean
    s2=subset_usv.sea_water_temperature_logger2_mean
    s3=subset_usv.sea_water_temperature_logger3_mean
    s4=subset_usv.sea_water_temperature_logger4_mean
    wnd=(subset_usv.UWND_MEAN**2+subset_usv.VWND_MEAN**2)**.5
    s0mask = np.isfinite(s0)
    s1mask = np.isfinite(s1)
    s2mask = np.isfinite(s2)
    s3mask = np.isfinite(s3)
    s4mask = np.isfinite(s4)
    wmask = np.isfinite(wnd)
    
    fig = plt.figure(figsize=(8,10))
    ax = fig.add_axes([0.1, 0.8, 0.8, 0.2])
    ax2 = ax.twinx()
    ax.plot_date(t[s1mask], s1[s1mask],'-',markersize=.5,lw=1)
    ax.plot_date(t[s0mask], s0[s0mask],'-',markersize=.5,lw=1)
    ax.plot_date(t[s2mask], s2[s2mask],'-',markersize=.5,lw=1)
    ax.plot_date(t[s3mask], s3[s3mask],'-',markersize=.5,lw=1)
    ax.plot_date(t[s4mask], s4[s4mask],'-',markersize=.5,lw=1)       
    ax.set_prop_cycle(None)
#    fig.gca().set_prop_cycle(None)   
    [y1,y2]=ax.get_ylim()
    [x1,x2]=ax.get_xlim()
    for ihr in range(3,24,3):
        tt=t[0]+np.timedelta64(ihr,'h')
        ax.plot([tt.data,tt.data],[y1,y2])
    ax2.plot_date(t[wmask], wnd[wmask],'k-',markersize=.5,lw=2)
    ax.set(xlabel='Date (LMT)', ylabel='SST (C)')
    ax2.set(ylabel='Wind Speed (ms$^{-1}$)')
    # rotate and align the tick labels so they look better
    fig.autofmt_xdate()
    ax.set_xlim(t[0].data,t[-1].data)
    #ax.set_xlim(subset)
    #ax.fmt_xdata = t.DateFormatter('%Y-%m-%d')
    tem=str(ds.TEMP_CTD_MEAN.installed_height)+' m'
    ax.legend([ds.sea_water_temperature_logger1_mean.installed_height,tem,ds.sea_water_temperature_logger2_mean.installed_height,ds.sea_water_temperature_logger3_mean.installed_height,ds.sea_water_temperature_logger4_mean.installed_height])
    ax.text(t[0].data+np.timedelta64(10,'m'),s1[s1mask].max().data,'A',size=12,weight='bold')

    #fig, ax = plt.subplots()
    
    profile_subset = profiles.sel(time=slice(i1,i2))
    profile_subset_lmt = profiles_lmt.sel(time=slice(i1,i2))
    fig.gca().set_prop_cycle(None)
    ax = fig.add_axes([0.1, 0.55, 0.8, 0.2])   
    x = profile_subset_lmt.data
    y = profile_subset.depth
    X, Y = np.meshgrid(x, y)
    c = profile_subset.values.T
    s = ax.contourf(X, Y, c, 8, cmap=cm.YlOrBr, origin='lower', vmax=1, vmin=0)
        # rotate and align the tick labels so they look better
    ax.set_ylabel('Depth (m)')
    [y1,y2]=ax.get_ylim()
    for ihr in range(3,24,3):
        tt=t[0]+np.timedelta64(ihr,'h')
        ax.plot([tt.data,tt.data],[y1,y2])
    ax.text(t[0].data+np.timedelta64(10,'m'),-0.4,'B',size=12,weight='bold')
    fig.autofmt_xdate()
    m = plt.cm.ScalarMappable(cmap=cm.YlOrBr)
    m.set_array(c)
    m.set_clim(0., 1.)
    colorbar_ax = fig.add_axes([0.905, 0.55, 0.015, 0.2])
    fig.colorbar(m, cax=colorbar_ax,orientation='vertical',label='$\Delta$SST [K]',ticks=[0,.25,.5,.75,1])
    #fig.set_label('$\Delta$SST [K]')
    ax = fig.add_axes([0.1, 0.3, 0.8, 0.2])
    ax0 = ax.pcolormesh(subset_adcp.tlmt,subset_adcp.cell_depth,subset_adcp_masked.vel_horiz.T,vmin=0,vmax=.5,cmap='seismic')
    #ax0.set_xlabel('Time')
    [y1,y2]=ax.get_ylim()
    for ihr in range(3,24,3):
        tt=t[0]+np.timedelta64(ihr,'h')
        ax.plot([tt.data,tt.data],[y1,y2])
    ax.set_ylabel('Depth (m)')
    ax.invert_yaxis()    
    ax.text(t[0].data+np.timedelta64(10,'m'),90,'C',size=12,weight='bold')
    
    colorbar_ax = fig.add_axes([0.905, 0.3, 0.015, 0.2])
    fig.colorbar(ax0, cax=colorbar_ax,orientation='vertical',label='Cur. Velocity (ms$^{-1}$)',ticks=[0,.5,1])

    ax2 = fig.add_axes([.1, 0.05, 0.4, 0.2])
    ilen_profile = profile_subset.shape[0]
    colors = plt.cm.jet(np.linspace(0,1,ilen_profile))
    for i in range(0,ilen_profile):
        ax0= plt.plot(profile_subset[i,:].values,z,color=colors[i])
    norm = mpl.colors.Normalize(vmin=6,vmax=30)
    cmap = colorbrewer.get_map('Spectral', 'diverging', 11, reverse=True).mpl_colormap
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    plt.colorbar(sm,boundaries=np.arange(6,30,.1),label='Hour')
   
#    for i in range(0,len(profile_subset.time)):
#        ax2.plot(profile_subset[i,:].values,z,'-',markersize=.5,lw=1)
#        ax2.plot(profile_subset[i,:].values,z,'.',markersize=.5,lw=1)
    ax2.set_xlabel('$\Delta$SST (K)')
    ax2.set_ylabel('Depth (m)')
    [x1,x2] = ax2.get_xlim()
    [y1,y2] = ax2.get_xlim()
    ax2.text(x1,-.35,'D',size=12,weight='bold')

    
    tem2=(subset_usv.time[0].values-np.datetime64('1970-01-01'))/np.timedelta64(1,'s')
    tem3=datetime.utcfromtimestamp(tem2)
    day_of_year = tem3.timetuple().tm_yday
   
    mn = xr.open_dataset(adir+'sat_data/virs_sst/all_sst'+str(day_of_year)+'.nc')
    mn.close()
#calculate overpass with most values
    subset_mn = mn.sel(lon=slice(subset_usv.longitude.min().data,subset_usv.longitude.max().data),
                       lat=slice(subset_usv.latitude.max().data,subset_usv.latitude.min().data))
    s=np.zeros(subset_mn.dims['time'])
    for i in range(0,subset_mn.dims['time']):
        s[i]=(subset_mn.sea_surface_temperature[i,:,:]/subset_mn.sea_surface_temperature[i,:,:]).sum()
    imax = np.argmax(s)
    subset_mn = mn.sel(lon=slice(subset_usv.longitude.min().data-.1,subset_usv.longitude.max().data+.1),
                       lat=slice(subset_usv.latitude.max().data+.1,subset_usv.latitude.min().data-.1))
    subset_mn = subset_mn.isel(time=imax)
#end calculate overpass with most values of sst

#    subset_mn = mn.sel(lon=slice(subset_usv.longitude.min().data-1,subset_usv.longitude.max().data+1),
#                       lat=slice(subset_usv.latitude.max().data+1,subset_usv.latitude.min().data-1))
    minsst = subset_mn.sea_surface_temperature.min().data-273.15
    maxsst = subset_mn.sea_surface_temperature.max().data-273.15
    print(minsst,maxsst)
    #blue marble image of ssts for cruise
    ax4 = fig.add_axes([.58, 0.05, 0.29, 0.2])    
    cmap = colorbrewer.get_map('Spectral', 'diverging', 11, reverse=True).mpl_colormap
    ax0 = ax4.pcolormesh(subset_mn.lon,subset_mn.lat, subset_mn.sea_surface_temperature - 273.15, cmap=cmap, vmin = minsst, vmax = maxsst)
    ax4.plot(subset_usv.longitude.data,subset_usv.latitude.data,'w', linewidth=3)
    ax4.plot(subset_usv.longitude.data,subset_usv.latitude.data,color='magenta', linewidth=1)
    for ihr in range(3,24,3):
        tt=subset_usv.time[0]+np.timedelta64(ihr,'h')
        sub_lons = subset_usv.longitude.sel(time=tt,method='nearest')
        sub_lats = subset_usv.latitude.sel(time=tt,method='nearest')
        ax4.plot(sub_lons,sub_lats,'o')
    #print('sub',sub_lons[0].data,sub_lats[0].data)
    ax4.text(subset_mn.lon.max().data-.04,subset_mn.lat.min().data+.02,'E',size=12,weight='bold')
    colorbar_ax = fig.add_axes([0.9, 0.05, 0.015, 0.2])
    fig.colorbar(ax0, cax=colorbar_ax,orientation='vertical',label='SST (C)')
#    ax0.colorbar(location="bottom",label='SST [C]') # draw colorbar
    adir_fig='F:/data/cruise_data/saildrone/baja-2018/figs/diurnal/'
    fig.savefig(adir_fig+'temp_loggers__diurnal_adcp'+str(day_of_year)+'3.png')
    


In [ ]:
sub_lons

In [ ]:
#old sst plotting code that used basemap which was nice because it has nice land
#but lacks lat / lon so the zoomed in images didn't tell a lot without location
m = Basemap(projection='merc', lat_0 = 27, lon_0 = -120,    resolution = 'l', area_thresh = 0.1,    
                                  llcrnrlon=min(subset_usv.longitude.data)-.05, llcrnrlat=min(subset_usv.latitude.data)-.05,
                urcrnrlon=max(subset_usv.longitude.data)+.05, urcrnrlat=max(subset_usv.latitude.data)+.05)
    m.bluemarble()
    #    m.fillcontinents(color='grey',lake_color='white')
    lon_grid, lat_grid = np.meshgrid(subset_mn.lon, subset_mn.lat)
    x,y = m(lon_grid,lat_grid)
    cs = m.pcolormesh(x,y, subset_mn.sea_surface_temperature - 273.15, cmap=cmap, vmin = minsst, vmax = maxsst)
    x,y = m(subset_usv.longitude.data,subset_usv.latitude.data)
    m.plot(x,y,'w', linewidth=3)
    m.plot(x,y,color='magenta', linewidth=1)
    for ihr in range(3,24,3):
        tt=subset_usv.time[0]+np.timedelta64(ihr,'h')
        sub_lons = subset_usv.longitude.sel(time=tt,method='nearest')
        sub_lats = subset_usv.latitude.sel(time=tt,method='nearest')
        x,y = m(sub_lons,sub_lats)
        m.plot(x,y,'o')
